In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow 
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Dropout, LeakyReLU, ReLU
from tensorflow.keras.optimizers import Adam
# import tensorflow_addons as tfa   ###  kaggle上不能用
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
import os
import csv
import numpy as np
from tensorflow.keras.models import load_model
import glob, tqdm
from PIL import Image

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [4]:
!nvidia-smi
# !pip list

Sat Oct 21 09:02:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
root_dir='/kaggle/input/taiwanese-food-101/tw_food_101/tw_food_101'
train_dir = root_dir + '/train'
test_dir = root_dir + '/test'

In [6]:
train_datagen = ImageDataGenerator(
        rotation_range=90,
        width_shift_range=0.2,
        height_shift_range=0.2,
        brightness_range=[0.,0.5],
        shear_range=0.2,
        zoom_range=0.2,
        channel_shift_range=0.2,
        fill_mode='nearest',
        horizontal_flip=True,
        vertical_flip=True,
        rescale=1./255,)

valid_datagen = ImageDataGenerator(
        # rotation_range=90,
        # width_shift_range=0.2,
        # height_shift_range=0.2,
        # brightness_range=[0.,0.5],
        # shear_range=0.2,
        # zoom_range=0.2,
        # channel_shift_range=0.2,
        # fill_mode='nearest',
        # horizontal_flip=True,
        # vertical_flip=True,
        rescale=1./255,)

In [7]:
train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(221, 221), 
    color_mode = "rgb",
    batch_size= 50,
    class_mode='categorical',
    shuffle = True)

Found 20372 images belonging to 101 classes.


In [8]:
valid_generator = valid_datagen.flow_from_directory(
    train_dir,
    target_size=(221, 221), 
    color_mode = "rgb",
    batch_size= 50,
    class_mode='categorical',
    shuffle = True)

Found 20372 images belonging to 101 classes.


In [9]:
base_model = keras.applications.densenet.DenseNet121(include_top=False, weights='imagenet', input_shape=(221, 221, 3))
# base_model = keras.applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', input_shape=(221, 221, 3))
#base_model = keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(150, 150, 3))
#base_model = keras.applications.resnet.ResNet50(include_top=False, weights='imagenet', input_shape=(150, 150, 3))
#base_model = keras.applications.mobilenet.MobileNet(include_top=False, weights='imagenet', input_shape=(150, 150, 3))

In [10]:
LR_function=ReduceLROnPlateau(monitor='val_accuracy',  # val_accuracy
                             patience=3,
                             # 3 epochs 內acc沒下降就要調整LR
                             verbose=1,
                             factor=0.5,
                             # LR降為0.5
                             min_lr=0.000001
                             # 最小 LR 到0.00001就不再下降
                             )
adam = Adam(learning_rate=0.0001)
estop = tensorflow.keras.callbacks.EarlyStopping(monitor='loss', patience=6)

scp = tensorflow.keras.callbacks.ModelCheckpoint(
    'model_DenseNet121.h5',
    monitor = "val_accuracy",
    verbose = 0,
    save_best_only = True,
    save_weights_only = False,
    mode = "auto",
    save_freq ="epoch",
)

In [11]:
# activation = LeakyReLU()

x = base_model.output
x = Flatten()(x)
x = Dense(2048, activation='relu')(x)  # , activation='relu'
# x = tfa.activations.mish(x)
x = Dense(1024, activation='relu')(x)  # , activation=mish  , activation='relu'
# x = tfa.activations.mish(x)
x = Dropout(.5)(x)
predictions = Dense(101, activation='softmax')(x)
model = Model(base_model.input, predictions)

In [12]:
model.compile(optimizer=adam,
            loss='categorical_crossentropy',
            metrics=['accuracy'])

In [13]:
history = model.fit(
      train_generator,
      epochs=100,
      callbacks=[LR_function, estop, scp],
      validation_data=valid_generator,
      validation_steps=None,
      validation_batch_size=None,
      validation_freq=1,
      workers=10
)

Epoch 1/100
 79/408 [====>.........................] - ETA: 2:53 - loss: 4.8684 - accuracy: 0.0316

/opt/conda/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:864: UserWarning: Truncated File Read
  warnings.warn(str(msg))


252/408 [=================>............] - ETA: 1:26 - loss: 3.9584 - accuracy: 0.1429

/opt/conda/lib/python3.10/site-packages/PIL/Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


408/408 [==============================] - 398s 782ms/step - loss: 3.4076 - accuracy: 0.2353 - val_loss: 1.5807 - val_accuracy: 0.5896 - lr: 1.0000e-04
Epoch 2/100
408/408 [==============================] - 321s 775ms/step - loss: 1.8926 - accuracy: 0.5207 - val_loss: 1.2284 - val_accuracy: 0.6665 - lr: 1.0000e-04
Epoch 3/100
408/408 [==============================] - 378s 916ms/step - loss: 1.4662 - accuracy: 0.6210 - val_loss: 1.0800 - val_accuracy: 0.7068 - lr: 1.0000e-04
Epoch 4/100
408/408 [==============================] - 320s 772ms/step - loss: 1.2477 - accuracy: 0.6788 - val_loss: 0.8448 - val_accuracy: 0.7718 - lr: 1.0000e-04
Epoch 5/100
408/408 [==============================] - 316s 764ms/step - loss: 1.0832 - accuracy: 0.7172 - val_loss: 0.8784 - val_accuracy: 0.7644 - lr: 1.0000e-04
Epoch 6/100
408/408 [==============================] - 315s 761ms/step - loss: 0.9756 - accuracy: 0.7449 - val_loss: 0.8157 - val_accuracy: 0.7810 - lr: 1.0000e-04
Epoch 7/100
408/408 [=======

In [14]:
model.save('model_DenseNet121.h5')

In [15]:
# Create {ID, filename} dictionary {'1269': '1269.jpg', '3863': '3863.jpg',.....}
test_dict = {}
for root, dirs, files in os.walk(root_dir + '/test'):
    for filename in files:
        test_id, file_ext = os.path.splitext(filename)
        test_dict[test_id] = filename

In [16]:
from PIL import Image
import numpy as np
from skimage import transform
def load_img(filename, target_w=150, target_h=150):
    np_image = Image.open(filename)
    np_image = np.array(np_image).astype('float32')/255
    np_image = transform.resize(np_image, (target_w, target_h, 3))
    np_image = np.expand_dims(np_image, axis=0)
    return np_image

In [17]:
# Read images in order and make predictions
results = []
for i in tqdm.tqdm(range(len(test_dict))):
    img = load_img(root_dir + '/test/' + test_dict[str(i)], 221, 221)
    ret = model.predict(img, verbose=0)
    results.append(np.argmax(ret))

100%|██████████| 5093/5093 [12:58<00:00,  6.54it/s]  


In [18]:
# Print results in CSV format and upload to Kaggle
with open('pred_results.csv', 'w') as f:
    f.write('Id,Category\n')
    for i in range(len(results)):
        f.write(str(i) + ',' + str(results[i]) + '\n')

In [19]:
# Download your results!
from IPython.display import FileLink
FileLink('pred_results.csv')

/kaggle/working/pred_results.csv